In [4]:
from kubernetes import client
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2TensorflowSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kubernetes.client import V1ResourceRequirements
import os
import sys
import argparse
import logging
import time

## kfserving module을 이용해 python code로 직접 클러스터에 serving을 하는 과정
class KFServing(object):
    def run(self):
        parser = argparse.ArgumentParser()
        parser.add_argument('--namespace', required=False, default='kubeflow')
        parser.add_argument('--storage_uri', required=False, default='/mnt/export')
        parser.add_argument('--name', required=False, default='kfserving-sample')        
        args = parser.parse_args()
        namespace = args.namespace
        serving_name =  args.name
        api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
        
        ## inference server spec 정의
        default_endpoint_spec = V1alpha2EndpointSpec(
                                  predictor=V1alpha2PredictorSpec(
                                    tensorflow=V1alpha2TensorflowSpec(
                                      storage_uri=args.storage_uri,
                                      resources=V1ResourceRequirements(
                                          requests={'cpu':'100m','memory':'1Gi'},
                                          limits={'cpu':'100m', 'memory':'1Gi'}))))
        ## kfserving crd(inferenceservice) 정의
        isvc = V1alpha2InferenceService(api_version=api_version,
                                  metadata=client.V1ObjectMeta(
                                      name=serving_name, namespace=namespace),
                                  spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))        
        
        KFServing = KFServingClient()
        KFServing.create(isvc)
        print('waiting 5 sec for Creating InferenceService')
        time.sleep(5)
        
        KFServing.get(serving_name, namespace=namespace, watch=True, timeout_seconds=300)

## serving하는 python 코드를 fairing으로 imaging 하는 작업
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow.fairing.builders.append.append import AppendBuilder
        from kubeflow.fairing.preprocessors.converted_notebook import \
            ConvertNotebookPreprocessor

        DOCKER_REGISTRY = 'rhojw'
        base_image = 'brightfly/kubeflow-kfserving:latest'
        image_name = 'kfserving'

        builder = AppendBuilder(
            registry=DOCKER_REGISTRY,
            image_name=image_name,
            base_image=base_image,
            push=True,
            preprocessor=ConvertNotebookPreprocessor(
                notebook_file="KFServing-fairing.ipynb"
            )
        )
        builder.build()
    else:
        serving = KFServing()
        serving.run()

[W 200415 10:00:25 append:50] Building image using Append builder...
[I 200415 10:00:25 base:105] Creating docker context: /tmp/fairing_context_e00hqtan
[I 200415 10:00:25 converted_notebook:127] Converting KFServing-fairing.ipynb to KFServing-fairing.py
[I 200415 10:00:25 docker_creds_:234] Loading Docker credentials for repository 'brightfly/kubeflow-kfserving:latest'
[W 200415 10:00:28 append:54] Image successfully built in 2.7561585219809785s.
[W 200415 10:00:28 append:94] Pushing image rhojw/kfserving:6962115B...
[I 200415 10:00:28 docker_creds_:234] Loading Docker credentials for repository 'rhojw/kfserving:6962115B'
[W 200415 10:00:28 append:81] Uploading rhojw/kfserving:6962115B
[I 200415 10:00:29 docker_session_:280] Layer sha256:fcb6b3d82de6c9870be99da3b3aba1ee9ae6fd451f58a349297c90d5a12620e1 exists, skipping
[I 200415 10:00:29 docker_session_:280] Layer sha256:d391d47713783ed4de6f856d63d4192240e9c93d782b158284aedc307cbe4fb6 exists, skipping
[I 200415 10:00:29 docker_session_